In [1]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
import cv2
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Input, InputLayer, Flatten
from tensorflow.keras.models import Sequential, Model
from  matplotlib import pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

In [2]:
IMAGE_SIZE=227
BATCH_SIZE=32
CHANNELS=3

In [3]:
dataset=tf.keras.preprocessing.image_dataset_from_directory(
    r"C:\Users\ronak\Downloads\archive\Driver Drowsiness Dataset (DDD)",
    shuffle=True,
    image_size=(IMAGE_SIZE,IMAGE_SIZE),
    batch_size=BATCH_SIZE
)

Found 41793 files belonging to 2 classes.


In [4]:
class_names=dataset.class_names
class_names

['Drowsy', 'Non Drowsy']

In [5]:
len(dataset)

1307

In [6]:
for image_batch,label_batch in dataset.take(1):
    print(image_batch.shape)
    print(label_batch.numpy())

(32, 227, 227, 3)
[1 1 0 1 1 1 0 0 1 0 0 1 1 1 1 0 1 1 1 0 0 0 1 1 0 1 1 1 0 0 1 0]


In [7]:
for image_batch,label_batch in dataset.take(1):
    print(image_batch[0])

tf.Tensor(
[[[178. 173. 164.]
  [178. 174. 165.]
  [177. 175. 166.]
  ...
  [171. 168. 159.]
  [170. 168. 159.]
  [169. 167. 158.]]

 [[178. 173. 164.]
  [177. 174. 165.]
  [177. 174. 166.]
  ...
  [172. 167. 159.]
  [171. 167. 159.]
  [170. 166. 158.]]

 [[178. 173. 164.]
  [176. 174. 164.]
  [175. 174. 164.]
  ...
  [172. 167. 159.]
  [172. 167. 159.]
  [171. 166. 158.]]

 ...

 [[137. 132. 124.]
  [139. 134. 129.]
  [142. 136. 134.]
  ...
  [255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]]

 [[136. 131. 124.]
  [139. 133. 129.]
  [141. 135. 133.]
  ...
  [255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]]

 [[137. 131. 126.]
  [138. 132. 129.]
  [140. 134. 132.]
  ...
  [255. 255. 255.]
  [255. 255. 255.]
  [255. 255. 255.]]], shape=(227, 227, 3), dtype=float32)


In [8]:
for image_batch,label_batch in dataset.take(1):
    print(image_batch[0].numpy())

[[[59. 41. 42.]
  [32. 20. 26.]
  [28. 17. 23.]
  ...
  [12. 10. 13.]
  [16. 11. 15.]
  [19. 15. 18.]]

 [[61. 44. 45.]
  [34. 23. 28.]
  [30. 18. 24.]
  ...
  [11.  9. 12.]
  [16. 11. 15.]
  [18. 13. 17.]]

 [[63. 46. 47.]
  [35. 24. 30.]
  [28. 17. 23.]
  ...
  [13.  9. 12.]
  [22. 17. 20.]
  [20. 16. 19.]]

 ...

 [[16. 22. 24.]
  [13. 16. 19.]
  [ 9. 11. 12.]
  ...
  [ 5.  5.  9.]
  [21. 22. 30.]
  [28. 32. 45.]]

 [[17. 19. 23.]
  [13. 18. 23.]
  [15. 17. 20.]
  ...
  [11. 14. 17.]
  [29. 31. 38.]
  [31. 35. 45.]]

 [[ 9. 11. 15.]
  [18. 23. 27.]
  [22. 24. 27.]
  ...
  [14. 16. 19.]
  [38. 39. 47.]
  [31. 35. 45.]]]


In [9]:
for image_batch,label_batch in dataset.take(1):
    print(image_batch[0].shape)

(227, 227, 3)


In [10]:
train_size=0.8
len(dataset)*train_size

1045.6000000000001

In [11]:
train_ds=dataset.take(1045)
len(train_ds)

1045

In [12]:
test_ds=dataset.skip(1045)
len(test_ds)

262

In [13]:
val_size=0.1
len(dataset)*val_size

130.70000000000002

In [14]:
val_ds=test_ds.take(130)
len(val_ds)

130

In [15]:
test_ds=test_ds.skip(130)
len(test_ds)

132

In [16]:
def get_dataset_partitions_tf(ds,train_split=0.8,val_split=0.1,test_split=0.1,shuffle=True,shuffle_size=10000):
    
    ds_size=len(ds)
    
    if shuffle:
        ds=ds.shuffle(shuffle_size,seed=12)
    
    train_size=int(train_split*ds_size)
    val_size=int(val_split*ds_size)

    train_ds=ds.take(train_size)
    
    val_ds=ds.skip(train_size).take(val_size)
    test_ds=ds.skip(train_size).skip(val_size)
    
    return train_ds,val_ds,test_ds

In [17]:
resize_and_rescale=tf.keras.Sequential([
    layers.experimental.preprocessing.Resizing(IMAGE_SIZE,IMAGE_SIZE),
    layers.experimental.preprocessing.Rescaling(1.0/255)
])

In [18]:
data_augmentation=tf.keras.Sequential([
    layers.experimental.preprocessing.RandomFlip('horizontal_and_vertical'),
    layers.experimental.preprocessing.RandomRotation(0.2)
])

In [19]:
from tensorflow.keras import models,layers
input_shape=(BATCH_SIZE,IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
n_classes=2

model=models.Sequential([
    resize_and_rescale,
    data_augmentation,
    layers.Conv2D(32,(3,3),activation='relu',input_shape=input_shape),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,kernel_size=(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,kernel_size=(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(64,activation='relu'),
    layers.Dense(n_classes,activation='softmax')
])

model.build(input_shape=input_shape)

In [20]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential (Sequential)      (32, 227, 227, 3)         0         
_________________________________________________________________
sequential_1 (Sequential)    (32, 227, 227, 3)         0         
_________________________________________________________________
conv2d (Conv2D)              (32, 225, 225, 32)        896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (32, 112, 112, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (32, 110, 110, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (32, 55, 55, 64)          0         
_________________________________________________________________
conv2d_2 (Conv2D)            (32, 53, 53, 64)         

In [21]:
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

In [22]:
history=model.fit(
    train_ds,
    epochs=5,
    batch_size=BATCH_SIZE,
    verbose=1,
    validation_data=val_ds
)

Epoch 1/5
1045/1045 [==============================] - 2296s 2s/step - loss: 0.6239 - accuracy: 0.6540 - val_loss: 0.5322 - val_accuracy: 0.7791
Epoch 2/5
1045/1045 [==============================] - 2075s 2s/step - loss: 0.6885 - accuracy: 0.6234 - val_loss: 0.8046 - val_accuracy: 0.5087
Epoch 3/5
1045/1045 [==============================] - 2359s 2s/step - loss: 0.8045 - accuracy: 0.5080 - val_loss: 0.8436 - val_accuracy: 0.4697
Epoch 4/5
1045/1045 [==============================] - 3226s 3s/step - loss: 0.8484 - accuracy: 0.4648 - val_loss: 0.8431 - val_accuracy: 0.4702
Epoch 5/5
1045/1045 [==============================] - 2353s 2s/step - loss: 0.8484 - accuracy: 0.4649 - val_loss: 0.8426 - val_accuracy: 0.4707


In [34]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.preprocessing.image import ImageDataGenerator

input_shape=(BATCH_SIZE,IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
n_classes=2

model = Sequential()
model.add(Conv2D(input_shape=(227, 227, 3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Flatten())
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(1, activation="sigmoid"))

In [35]:
from keras.optimizers import Adam
opt = Adam(lr=0.001)
model.compile(optimizer=opt, loss=keras.losses.binary_crossentropy, metrics=['accuracy'])

In [36]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_34 (Conv2D)           (None, 227, 227, 64)      1792      
_________________________________________________________________
conv2d_35 (Conv2D)           (None, 227, 227, 64)      36928     
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 113, 113, 64)      0         
_________________________________________________________________
conv2d_36 (Conv2D)           (None, 113, 113, 128)     73856     
_________________________________________________________________
conv2d_37 (Conv2D)           (None, 113, 113, 128)     147584    
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 56, 56, 128)       0         
_________________________________________________________________
conv2d_38 (Conv2D)           (None, 56, 56, 256)      

In [37]:
history=model.fit(
    train_ds,
    epochs=5,
    batch_size=None,
    verbose=1,
    validation_data=val_ds
)

Epoch 1/5
   3/1045 [..............................] - ETA: 4:37:08 - loss: 5450.0747 - accuracy: 0.5729

KeyboardInterrupt: 